<a href="https://colab.research.google.com/github/mcstllns/DeepLearning_2025/blob/main/PRACTICA_05_Ajuste_de_la_red.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="darkorange" size="10"><b>05. Ajuste de la red</b></font>

Miguel A. Castellanos

# Estocástico, Batch y mini-batch

Esto define si queremos un aprendizaje estocástico (actualización cada sujeto), por batch (actualización con el promedio de todos los sujetos) o por mini-batch (actualización por lotes de sujetos).

Se determina simplemente usando la función DataLoader

In [ ]:
dataloader = DataLoader(dataset) # estocastico
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # mini-batch
dataloader = DataLoader(dataset, batch_size=len(my_dataset), shuffle=True) # batch completo


# Optimizadores

## SGD


```python
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4, nesterov=True)
```

Siendo:
- lr: Learning rate
- momentum: Beta de momentum
- weight_decay: el lambda2 de la Regularización L2
- nesterov: Si está en True se activa la aceleración de nesterov

### Aceleración de Nesterov
Cuando activas nesterov=True, el optimizador ajusta el cálculo del momentum para "mirar hacia adelante" antes de actualizar los pesos. Esto puede hacer que el entrenamiento sea más estable y rápido. Requiere que el momentum sea mayor que 0 (momentum > 0), de lo contrario, no tiene efecto.

SGD hará aprendizaje estocástico si no definimos mini-batch. Si hacemos n=n1 o n=N hará mini-batchs o batch

Básico y funciona bien. Muy sensible al Learning Rate.


## RMSprop

```python
optimizer = optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-8, weight_decay=0, momentum=0.9)
```
- lr:	Tasa de aprendizaje. Controla el tamaño de los pasos en la actualización de pesos.
- alpha:	Factor de suavizado (0.99 por defecto). Controla cuánto contribuyen los gradientes pasados a la media móvil. Es lo que hemos llamado Beta en teoría
- eps:	Término de estabilidad. Evita divisiones por cero (típicamente un valor muy muy pequeño, 1e-8). Es l oque hemos llamado epsilon en teoría.
- weight_decay:	Regularización L2. Similar a la regularización en SGD, evita sobreajuste.
- momentum:	Añade momento (por defecto 0). Se puede usar para mejorar convergencia.
- centered:	Si es True, usa una media móvil del gradiente para normalizar mejor. No se usa mucho.

Funciona bien en problemas longitudinales como redes recurrentes (RNNs).
No siempre converge a la mejor solución (a veces encuentra mínimos locales).
Menos robusto en redes muy profundas comparado con Adam.


## Adam

```python
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-4)
```

- lr:	Tasa de aprendizaje (0.001 por defecto).
- betas:	Factores de decaimiento para momentum beta1 (0.9) y el segundo beta2 controla controla el beta del RMSProp
- eps:	Término de estabilidad (1e-8). Evita divisiones por cero.
- weight_decay:	Regularización L2 (por defecto 0)

Es el que mejor funciona porque combina todo lo anterior. Requiere mas memoria que los anteriores y más cálculo. Hay que elegir sabiamente las betas para que vaya bien.

# Inicializacion de parametros

Por defecto utiliza la siguiente inicialización y funciona muy bien, la verdad.


| Capa | Inicialización por defecto |
|------|----------------------------|
|nn.Linear |	Uniforme en [-√k, √k], donde k = 1/in_features|
| nn.Conv2d	| Uniforme en [-√k, √k], donde k = 1/in_features|
| nn.BatchNorm	| pesos = 1, sesgos = 0 |

Si se quiere utilizar las inicializaciones de Xavier, He y LeCun puede hacerse lo siguiente:




In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

class MLP(nn.Module):
    def __init__(self, X_nvars):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # Hidden Layer 1
            torch.nn.Linear(X_nvars, 1), # Hacemos el sumatorio
            torch.nn.ReLU(),              # Aplicamos la función de activacion

            # output layer
            torch.nn.Linear(1, 1),
        )

        # Aqui es donde hacemos las incializaciones
        for layer in self.layers:
          if isinstance(layer, nn.Linear):
            init.xavier_uniform_(layer.weight)# Si queremos xavier con uniforme
            # init.xavier_normal_(layer.weight) # Si queremos xavier con normal

            # init.kaiming_uniform_(layer.weight, nonlinearity='relu') # Si queremos He con uniforme
            # init.kaiming_normal_(layer.weight, nonlinearity='relu') # Si queremos He con normal

            # # LeCun normal
            # init.normal_(layer.weight, mean=0, std=(1.0 / torch.sqrt(torch.tensor(layer.weight.size(1), dtype=torch.float))))

            # # Inicialización LeCun uniforme (no hay una función directa, pero se puede hacer manualmente)
            # fan_in = layer.weight.size(1)  # Número de entradas
            # bound = 1 / torch.sqrt(torch.tensor(fan_in, dtype=torch.float))
            # init.uniform_(layer.weight, -bound, bound)

    def forward(self, x):
        output = self.layers(x)
        return output

# Crear conjuntos de train, dev y test

Lo primero, hay que tener en cuenta estas dos funciones:

```python
model.train()
model.eval()
```

Ponen al modelo en modo entrenamiento o de evaluación y tiene efecto sobre algunas cosas del funcionamiento de la red, por ejemplo si se hacen o no los dropout, si se calcula el gradiente, etc.

La división entre los conjuntos de datos se puede hacer cómo quieras, hay múltiples funciones para ello. Pytorch incorpora el random_split


In [ ]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split

X = torch.randn(1000, 10)
y = torch.randint(0, 2, (1000,))  # Clasificación binaria

dataset = TensorDataset(X, y) # create your datset

# Definir tamaños de cada conjunto
train_size = int(0.8 * len(dataset))  # 80% para entrenamiento
dev_size = int(0.1 * len(dataset))  # 10% para validación
test_size = len(dataset) - train_size - dev_size  # 10% para prueba

# Dividir aleatoriamente
train_dataset, dev_dataset, test_dataset = random_split(dataset, [train_size, dev_size, test_size])

# Crear dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dev_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


# Dropout

Simplemente se introduce una capa de dropout después de la activación. El parámetro p define la cantidad de datos que queremos perder.

- Si lo aplicamos antes de la activación, algunas neuronas pueden ser eliminadas antes de calcular su valor útil, perdiendo información antes de que sea procesada correctamente.
- Aplicarlo después de la activación asegura que cada neurona ya tiene su contribución calculada antes de ser descartada temporalmente.


In [ ]:
# dropout_p es la probabilidad de apagar una neurona (50%)
dropout_p = 0.5

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size, dropout_p):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            # Capa de entrada a capa oculta 1
            nn.Linear(input_size, hidden_size_1),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),  # Dropout después de la capa oculta 1

            # Capa oculta 1 a capa oculta 2
            nn.Linear(hidden_size_1, hidden_size_2),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),  # Dropout después de la capa oculta 2

            # Capa oculta 2 a capa de salida
            nn.Linear(hidden_size_2, output_size),
            nn.Sigmoid()  # Para clasificación binaria, puedes usar softmax para clasificación múltiple
        )

    def forward(self, x):
        return self.model(x)


# Batch-normalization

Batch Normalization es una técnica para normalizar (media = 0 y sd = 1) las activaciones de cada capa en mini-batches. Esto ayuda a acelerar el entrenamiento y a estabilizar la red. En PyTorch, puedes usar torch.nn.BatchNorm1d, BatchNorm2d, o BatchNorm3d, dependiendo de la dimensión de los datos que estés manejando.


Ademas, la función BatchNorm de pytorch introduce dos nuevos parámetros:

- β (bias): permite un desplazamiento en la media.
- γ (scale factor): escala la distribución normalizada.

Es decir, se le puede decir que en vez de acabar en (0,1) se reescale a y (β,γ). Esto permite que la red aprenda si necesita reescalar o desplazar los valores.

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            # Capa de entrada a capa oculta 1
            nn.Linear(input_size, hidden_size_1),
            nn.BatchNorm1d(hidden_size_1),  # Normalización después de la capa 1
            nn.ReLU(),

            # Capa oculta 1 a capa oculta 2
            nn.Linear(hidden_size_1, hidden_size_2),
            nn.BatchNorm1d(hidden_size_2),  # Normalización después de la capa 2
            nn.ReLU(),

            # Capa oculta 2 a capa de salida
            nn.Linear(hidden_size_2, output_size),
            nn.Sigmoid()  # Para clasificación binaria
        )

    def forward(self, x):
        return self.model(x)

# Regularizaciones L1 y L2

Lo más eficiente es usar L2 con el weight dacay de los optimizadores, pero si quieres programarla a pelo, o programar una elasticNet, puedes hacerlo:

In [ ]:
# Se define ElasticNet
# Si haces l1_lambda o l2_lambda = 0 ese parámetro no influye
def l1_l2_regularization(model, l1_lambda, l2_lambda):
    l1_norm = sum(p.abs().sum() for p in model.parameters())  # L1
    l2_norm = sum(p.pow(2).sum() for p in model.parameters())  # L2
    return l1_lambda * l1_norm + l2_lambda * l2_norm

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, targets) in enumerate(my_dataloader):

        # forward
        output = model(features)
        loss = loss_fn(output, targets)

        # La regularización
        reg_loss = l1_l2_regularization(model, l1_lambda, l2_lambda)
        total_loss = loss + reg_loss

        # backward
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()


# Usar la GPU

Lo primero es crear un entorno de ejecución con una GPU

Conectar -> Cambiar tipo de entorno de ejecución

Y elegir GPU-T4

In [ ]:
# Comprobamos si tenemos GPU
import torch
print(torch.cuda.is_available())  # Salida: True si hay una GPU disponible

In [ ]:
# Si hay cuda indicamos que la GPU va a ser nuestro dispositivo de calculo, si no, la CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Si tentemos cuda ponemos imprimir informacion
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


In [ ]:
# Esta función nos dice cosas de la Tarjeta grafica

if device.type == 'cuda':
  !nvidia-smi

In [ ]:
# Como lanzar un MLP sobre la GPU

# Lo primero es pasar todos los tensores a la GPU

# Dependiendo de la cantidad de memoria se recomiendan dos opciones:
# A. Pasamos todos los datos a la GPU desde el principio (si tenemos mucha Vram)

X, y = X.to(device), y.to(device)

# DataLoader sin necesidad de mover lotes a GPU
dataloader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)

# B. Vamos pasando los mini-batch según los vayamos usando
for epoch in range(5):
    for x_batch, y_batch in dataloader:

        # Mover solo el batch actual a GPU
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)


# Lo segundo ejecutar el modelo sobre la GPU
model = MLP().to(device)

# Y el resto del código sería igual que en otras ocasiones

# Búsqueda de hiper-parámetros con Optuna

Vamos a partir de un MLP que ya vimos en la PRACTICA 04. Una red para clasificación binaria


In [ ]:
!pip install torcheval
!pip install optuna

In [ ]:
# Creamos unos datos

import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import torcheval
from torch.utils.data import DataLoader, TensorDataset
from torcheval.metrics import BinaryAccuracy

X = torch.randn(1000, 10)
y = torch.randint(0, 2, (1000,))  # Clasificación binaria

train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [ ]:
# Se define el MLP

class MLP(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_features, 25),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(25, 1),
            # torch.nn.Sigmoid() # ver comentario 1

        )

    def forward(self, x):
        output = self.all_layers(x)
        return output.flatten()



In [ ]:
# Se define Optuna

def objective(trial):

    # Aqui es donde se especifican los parametros que vamos a optimizar
    # Vamos a optimizar solo el lr del optimizador
    learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.01)

    # Crear modelo con los hiperparámetros sugeridos
    model = MLP(num_features = 10)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    accuracy = BinaryAccuracy()

    model.train()
    # Entrenamiento simple
    for epoch in range(5):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = loss_fn(output, y_batch.float())
            loss.backward()
            optimizer.step()

    # Evaluar precisión
    model.eval()
    for X_batch, y_batch in train_loader:
        output = model(X_batch)
        accuracy.update(output, y_batch)

    return accuracy.compute()  # Optuna intentará maximizar esto


In [ ]:
study = optuna.create_study(direction="maximize")  # Buscamos maximizar la precisión
study.optimize(objective, n_trials=20)  # Probar 20 combinaciones

# 🔹 Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", study.best_params)

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(study)

In [ ]:
optuna.visualization.matplotlib.plot_optimization_history(study)

### Ejercicio

Para esta práctica vamos a utilizar el conjunto de datos MNIST que consiste en imágenes de números del 0 al 9 escritas a mano y digitalizadas. Una descripción del fichero la puedes encontrar en la [wikipedia](https://en.wikipedia.org/wiki/MNIST_database).

Los datos están ya almacenados en pytorch, con lo que con una única función los podemos cargar y, además, están ya divididos en dos conjuntos de datos, un de entrenamiento con 60000 imágenes y otro de test con 10000 imágenes.

El objetivo de la red es ser capaz de identificar correctamente el número y eso es equivalente a clasificar correctamente cada imagen.

Los datos de entrada son imágenes en escala de grises, una matriz bidimensional de 28 x 28 en la que cada pixel va de 0 a 255. La mejor manera de trabajar con imágenes es a través de redes convolucionales pero todavía no las hemos estudiado así que vamos a vectorizar la imagen, es decir, concatenar las columnas una tras otra y formas un unico vector para cada imagen.

Para procesar los datos vamos a construir un perceptron multicapa como los de los ejercicios anteriores pero ahora como entrada vamos a tener

- **Para el train:** 60000 imágenes y sus etiquetas
- **Para el test:** 10000 imágenes y sus etiquetas


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [ ]:
class Flatten(object):
    def __call__(self, tensor):
        # Aplanar la imagen a un vector de 784 píxeles
        return tensor.view(-1)


transform = transforms.Compose([
    transforms.ToTensor(),  # Convierte la imagen a tensor
    transforms.Normalize((0.5,), (0.5,)),  # Normalización
    transforms.Lambda(lambda x: x.view(-1)) # Aplanar las imágenes a un vector de 784 píxeles
    # Flatten()  # Aplanar las imágenes a un vector de 784 píxeles
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


In [ ]:
# Obtener una imagen y su etiqueta
image, label = train_dataset[5]  # Obtener la primera imagen del conjunto de entrenamiento

# Visualizar la imagen
plt.imshow(image.view(28,28), cmap='gray')  # .squeeze() elimina la dimensión de los canales (1,28,28 -> 28,28)
plt.title(f"Etiqueta: {label}")
plt.show()

### Ejercicio 01

Crea un MLP que realice una clasificación razonable, para ello utiliza las estrategias ***que consideres oportunas***.

- Crear conjuntos de entrenamiento, dev y test
- Ejecuta la red sobre una GPU
- Buscar parámetros con Optuna
- Usar regularizaciones
- etc.